# Clothing Review Rating Prediction

# Importing the Usuall Libary

In [101]:
import numpy as np
import pandas as pd
#train_test_split
from sklearn.model_selection import train_test_split
#Tokenizer from keras
from tensorflow.keras.preprocessing.text import Tokenizer
#pad_seuences
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

# Loading the Dataset

In [102]:
df=pd.read_csv('/kaggle/input/womens-ecommerce-clothing-reviews/Womens Clothing E-Commerce Reviews.csv')
#showing the dataset
df

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
...,...,...,...,...,...,...,...,...,...,...,...
23481,23481,1104,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0,General Petite,Dresses,Dresses
23482,23482,862,48,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0,General Petite,Tops,Knits
23483,23483,1104,31,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1,General Petite,Dresses,Dresses
23484,23484,1084,28,"Very cute dress, perfect for summer parties an...",I bought this dress for a wedding i have this ...,3,1,2,General,Dresses,Dresses


# Checking for Preliminary Information

In [103]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486 entries, 0 to 23485
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0               23486 non-null  int64 
 1   Clothing ID              23486 non-null  int64 
 2   Age                      23486 non-null  int64 
 3   Title                    19676 non-null  object
 4   Review Text              22641 non-null  object
 5   Rating                   23486 non-null  int64 
 6   Recommended IND          23486 non-null  int64 
 7   Positive Feedback Count  23486 non-null  int64 
 8   Division Name            23472 non-null  object
 9   Department Name          23472 non-null  object
 10  Class Name               23472 non-null  object
dtypes: int64(6), object(5)
memory usage: 2.0+ MB


# Preprocessing 

In [104]:
#creating the function
def preprocess_inputs(df):
    df=df.copy()
    #dropping the columns with missing reviews
    missing_reviews_rows=df[df['Review Text'].isna()].index
    df=df.drop(missing_reviews_rows,axis=0).reset_index(drop=True)
    #splitting the data between feature and target
    y=df['Rating']
    x=df['Review Text']
    y=y.apply(lambda x:1 if x==5 else 0)
    #train_test_split
    x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.7,
                                                  shuffle=True,random_state=1)
    
    #Learn the vocabulary
    tokenizer=Tokenizer()
    tokenizer.fit_on_texts(x_train)
    
    vocal_length=len(tokenizer.word_index)+1
    
    #Convert review text into sequences of integers
    x_train=tokenizer.texts_to_sequences(x_train)
    x_test=tokenizer.texts_to_sequences(x_test)
    
    #Finding the maximum sequence length
    max_sequence_length=np.max(list(map(lambda x:len(x),x_train)))
    x_train=pad_sequences(x_train,max_sequence_length,padding='post')
    x_test=pad_sequences(x_test,max_sequence_length,padding='post')
    return x_train,x_test,y_train,y_test,tokenizer

In [105]:
np.max(list(map(lambda x:len(x),x_train)))

116

# Training the Model

In [ ]:
inputs=tf.keras.Input(shape=(x_train.shape[1],))
x=tf.keras.layers.Embedding(
    input_dim,
    output_dim,
    input_length)

In [106]:
x_train,x_test,y_train,y_test,t=preprocess_inputs(df)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(15848, 116)
(6793, 116)
(15848,)
(6793,)


In [112]:
x_test

array([[  36, 1080,    7, ...,    0,    0,    0],
       [   2,  141,    7, ...,    0,    0,    0],
       [ 261,    9,   22, ...,    0,    0,    0],
       ...,
       [   2,   92,    1, ...,    0,    0,    0],
       [  52,  185,    1, ...,    0,    0,    0],
       [  39,  141,    7, ...,    0,    0,    0]], dtype=int32)

In [108]:
x_train

array([[  7,  18,   6, ...,   0,   0,   0],
       [  1,  26,  15, ...,   0,   0,   0],
       [  2,  74,   7, ...,   0,   0,   0],
       ...,
       [349,  23,  52, ...,   0,   0,   0],
       [  2,  50,  28, ...,   0,   0,   0],
       [ 22, 515,  22, ...,   0,   0,   0]], dtype=int32)

In [109]:
df[df['Review Text'].isna()].index

Int64Index([   92,    93,    98,   135,   142,   165,   168,   192,   201,
              248,
            ...
            23080, 23120, 23127, 23166, 23224, 23258, 23301, 23303, 23470,
            23480],
           dtype='int64', length=845)

In [110]:
y

0        4
1        5
2        3
3        5
4        5
        ..
22636    5
22637    3
22638    3
22639    3
22640    5
Name: Rating, Length: 22641, dtype: int64